<a href="https://colab.research.google.com/github/Balaji-A-2823/Machine_Learning_By_Providing_Dataset/blob/main/svm_reg_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
try:
    df = pd.read_csv('/content/svm_project/MW_Daily.csv')
    print("Dataset loaded successfully:")
    display(df.head())
except FileNotFoundError:
    print("Error: '/content/svm_project/MW_Daily.csv' not found. Please upload your dataset file or provide the correct path.")
except Exception as e:
    print(f"An error occurred while loading the dataset: {e}")

# Assuming your dataset has features and a target variable, you would separate them like this:
# Replace 'target_column_name' with the name of your target column
# Replace 'feature_column_names' with a list of your feature column names or drop the target column from df
# X_custom = df[['feature_column_1', 'feature_column_2', ...]]
# y_custom = df['target_column_name']

# Or, if the target is the last column and all others are features:
# X_custom = df.iloc[:, :-1]
# y_custom = df.iloc[:, -1]

# Now you can use X_custom and y_custom in the subsequent steps of splitting, scaling, and training the SVM model.

Dataset loaded successfully:


,date,year,month,day,n_mw,w_mw,s_mw,e_mw,ne_mw,ind_mw
0,01-01-2014,2014,Jan,WED,35136,36914,31623,14794,1761,120228
1,02-01-2014,2014,Jan,THU,36462,38019,32126,15083,1797,123487
2,03-01-2014,2014,Jan,FRI,37787,37776,32573,14592,1861,124589
3,04-01-2014,2014,Jan,SAT,36636,36784,31636,14804,1804,121664
4,05-01-2014,2014,Jan,SUN,35764,36419,29973,14118,1798,118072


In [2]:
variable_name = "" # @param {"type":"string"}
# SVM Power Generation Analysis - Quick Start Guide
# This script demonstrates the basic usage of the SVM implementation

import pandas as pd
import numpy as np
from sklearn.svm import SVR, SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report

def quick_svm_analysis(csv_file='/content/svm_project/MW_Daily.csv'):
    """
    Quick SVM analysis for power generation data

    This function demonstrates:
    1. Data loading and preprocessing
    2. SVM regression for total power prediction
    3. SVM classification for demand level prediction
    4. Model evaluation and predictions
    """

    print("="*60)
    print("SVM POWER GENERATION ANALYSIS - QUICK START")
    print("="*60)

    # Step 1: Load and preprocess data
    print("1. Loading and preprocessing data...")

    df = pd.read_csv(csv_file)
    print(f"   Dataset shape: {df.shape}")
    print(f"   Date range: {df['date'].iloc[0]} to {df['date'].iloc[-1]}")

    # Convert date and create features
    df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')
    df['month_num'] = df['date'].dt.month
    df['weekday'] = df['date'].dt.weekday
    df['quarter'] = df['date'].dt.quarter

    # Create demand categories
    q1, q2 = df['ind_mw'].quantile([0.33, 0.67])
    conditions = [
        df['ind_mw'] <= q1,
        (df['ind_mw'] > q1) & (df['ind_mw'] <= q2),
        df['ind_mw'] > q2
    ]
    choices = ['Low', 'Medium', 'High']
    df['demand_class'] = np.select(conditions, choices, default='Unknown') # Added default string value

    print(f"   Created {len(df.columns)} features")

    # Step 2: Prepare features
    print("2. Preparing features...")

    features = ['n_mw', 'w_mw', 's_mw', 'e_mw', 'ne_mw',
                'year', 'month_num', 'weekday', 'quarter']

    X = df[features]
    y_regression = df['ind_mw']  # Total power (continuous)
    y_classification = df['demand_class']  # Demand level (categorical)

    print(f"   Using {len(features)} features: {features}")

    # Step 3: Train SVM Regression Model
    print("3. Training SVM Regression model...")

    # Split data for regression
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
        X, y_regression, test_size=0.2, random_state=42
    )

    # Scale features
    scaler_reg = StandardScaler()
    X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
    X_test_reg_scaled = scaler_reg.transform(X_test_reg)

    # Train regression model
    svr_model = SVR(C=100, kernel='rbf', gamma='scale')
    svr_model.fit(X_train_reg_scaled, y_train_reg)

    # Evaluate regression
    y_pred_reg = svr_model.predict(X_test_reg_scaled)
    r2_reg = r2_score(y_test_reg, y_pred_reg)
    rmse_reg = np.sqrt(mean_squared_error(y_test_reg, y_pred_reg))
    mae_reg = np.mean(np.abs(y_test_reg - y_pred_reg))

    print(f"   Regression Results:")
    print(f"   - R² Score: {r2_reg:.4f} ({r2_reg*100:.1f}% of variance explained)")
    print(f"   - RMSE: {rmse_reg:.0f} MW")
    print(f"   - MAE: {mae_reg:.0f} MW")

    # Step 4: Train SVM Classification Model
    print("4. Training SVM Classification model...")

    # Split data for classification
    X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
        X, y_classification, test_size=0.2, random_state=42, stratify=y_classification
    )

    # Scale features
    scaler_clf = StandardScaler()
    X_train_clf_scaled = scaler_clf.fit_transform(X_train_clf)
    X_test_clf_scaled = scaler_clf.transform(X_test_clf)

    # Train classification model
    svc_model = SVC(C=10, kernel='linear', random_state=42)
    svc_model.fit(X_train_clf_scaled, y_train_clf)

    # Evaluate classification
    y_pred_clf = svc_model.predict(X_test_clf_scaled)
    accuracy_clf = accuracy_score(y_test_clf, y_pred_clf)

    print(f"   Classification Results:")
    print(f"   - Accuracy: {accuracy_clf:.4f} ({accuracy_clf*100:.1f}%)")
    print(f"   - Classes: {sorted(df['demand_class'].unique())}")


    # Step 5: Make Predictions on Sample Data
    print("5. Making predictions on sample data...")

    # Create sample data
    sample_data = pd.DataFrame({
        'n_mw': [45000, 55000, 40000],
        'w_mw': [46000, 48000, 42000],
        's_mw': [38000, 41000, 35000],
        'e_mw': [19000, 21000, 17000],
        'ne_mw': [2500, 2800, 2200],
        'year': [2021, 2021, 2021],
        'month_num': [6, 6, 12],
        'weekday': [1, 3, 5],
        'quarter': [2, 2, 4]
    })

    # Scale sample data
    sample_scaled_reg = scaler_reg.transform(sample_data[features])
    sample_scaled_clf = scaler_clf.transform(sample_data[features])

    # Make predictions
    total_power_pred = svr_model.predict(sample_scaled_reg)
    demand_level_pred = svc_model.predict(sample_scaled_clf)

    print("   Sample Predictions:")
    for i in range(len(sample_data)):
        regional_sum = (sample_data.iloc[i]['n_mw'] + sample_data.iloc[i]['w_mw'] +
                       sample_data.iloc[i]['s_mw'] + sample_data.iloc[i]['e_mw'] +
                       sample_data.iloc[i]['ne_mw'])

        print(f"   Sample {i+1}:")
        print(f"     Regional Power: {regional_sum:,} MW")
        print(f"     Predicted Total: {total_power_pred[i]:,.0f} MW")
        print(f"     Predicted Level: {demand_level_pred[i]}")
        print()


    # Step 6: Summary
    print("6. Summary:")
    print("   ✓ SVM Regression model successfully predicts total power generation")
    print("   ✓ SVM Classification model accurately categorizes demand levels")
    print("   ✓ Both models show excellent performance on test data")

    # Return trained models for further use
    models = {
        'regression': {
            'model': svr_model,
            'scaler': scaler_reg,
            'features': features,
            'metrics': {'r2': r2_reg, 'rmse': rmse_reg, 'mae': mae_reg}
        },
        'classification': {
            'model': svc_model,
            'scaler': scaler_clf,
            'features': features,
            'metrics': {'accuracy': accuracy_clf}
        }
    }

    print("\n" + "="*60)
    print("ANALYSIS COMPLETED SUCCESSFULLY!")
    print("="*60)

    return models


def predict_new_data(models, new_data):
    """
    Use trained models to predict on new data

    Parameters:
    - models: Dictionary with trained models from quick_svm_analysis()
    - new_data: DataFrame with required feature columns

    Returns:
    - DataFrame with predictions
    """

    features = models['regression']['features']

    # Regression prediction
    reg_scaler = models['regression']['scaler']
    reg_model = models['regression']['model']
    X_reg_scaled = reg_scaler.transform(new_data[features])
    total_power_pred = reg_model.predict(X_reg_scaled)

    # Classification prediction
    clf_scaler = models['classification']['scaler']
    clf_model = models['classification']['model']
    X_clf_scaled = clf_scaler.transform(new_data[features])
    demand_level_pred = clf_model.predict(X_clf_scaled)

    # Combine results
    predictions = pd.DataFrame({
        'predicted_total_power': total_power_pred,
        'predicted_demand_level': demand_level_pred
    })

    return predictions

# Example usage
if __name__ == "__main__":
    try:
        # Run the quick analysis
        print("Starting SVM Power Generation Analysis...")
        trained_models = quick_svm_analysis("/content/svm_project/MW_Daily.csv")

        # Your existing prediction code...
        if trained_models: # Check if trained_models was successfully created
            new_scenarios = pd.DataFrame({
                'n_mw': [50000, 60000],
                'w_mw': [48000, 52000],
                's_mw': [42000, 46000],
                'e_mw': [20000, 24000],
                'ne_mw': [2600, 2900],
                'year': [2024, 2024],
                'month_num': [7, 8],
                'weekday': [0, 2],
                'quarter': [3, 3]
            })

            predictions = predict_new_data(trained_models, new_scenarios)

            print("New Scenario Predictions:")
            for i in range(len(new_scenarios)):
                print(f"Scenario {i+1}: Total Power = {predictions.iloc[i]['predicted_total_power']:,.0f} MW, "
                      f"Level = {predictions.iloc[i]['predicted_demand_level']}")


    except FileNotFoundError:
        print("Error: MW_Daily.csv file not found!")
        print("Please ensure the data file is in the same directory as this script.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

Starting SVM Power Generation Analysis...
SVM POWER GENERATION ANALYSIS - QUICK START
1. Loading and preprocessing data...
   Dataset shape: (2742, 10)
   Date range: 01-01-2014 to 04-07-2021
   Created 14 features
2. Preparing features...
   Using 9 features: ['n_mw', 'w_mw', 's_mw', 'e_mw', 'ne_mw', 'year', 'month_num', 'weekday', 'quarter']
3. Training SVM Regression model...
   Regression Results:
   - R² Score: 0.9006 (90.1% of variance explained)
   - RMSE: 4523 MW
   - MAE: 2960 MW
4. Training SVM Classification model...
   Classification Results:
   - Accuracy: 0.9945 (99.5%)
   - Classes: ['High', 'Low', 'Medium']
5. Making predictions on sample data...
   Sample Predictions:
   Sample 1:
     Regional Power: 150,500 MW
     Predicted Total: 154,457 MW
     Predicted Level: Medium

   Sample 2:
     Regional Power: 167,800 MW
     Predicted Total: 163,942 MW
     Predicted Level: High

   Sample 3:
     Regional Power: 136,200 MW
     Predicted Total: 141,304 MW
     Predicted